In [1]:
from datetime import datetime
import pandas as pd

In [2]:
df_inv = pd.read_excel('../../data/raw/INV_Transacton_Summary.xlsx', engine='openpyxl', sheet_name='Select dual')

## Data Preprosessing
PO_RECEIPT=0이 기록되지 않은 시점에도 0 기록

In [3]:
""" SUBINV이 한 IEM_CODE에서도 다르게 발생 -> 발주가 들어가지 않은 경우 그냥 0으로 처리하고 진행 """

# item별 full date dataframe 생성

full_date = pd.DataFrame(df_inv['YYYYMM'].value_counts().sort_index().index, columns=['ds'])
cols_item = ['IEM_CODE', 'ITEM_DESCRIPTION', 'SPG', 'UOM']
df_item = df_inv[cols_item].drop_duplicates().reset_index(drop=True)

from itertools import product
tmp = pd.DataFrame(product(full_date['ds'], df_item['IEM_CODE']), columns=['YYYYMM', 'IEM_CODE']) # item code별 full date dataframe 생성
df_full_date_item = pd.merge(tmp, df_item, on='IEM_CODE')
print(f'Length check : {len(full_date)*len(df_item)} == {len(df_full_date_item)}')
df_full_date_item

Length check : 71295 == 71295


,YYYYMM,IEM_CODE,ITEM_DESCRIPTION,SPG,UOM
0,201401,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",500,ea
1,201402,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",500,ea
2,201403,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",500,ea
3,201404,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",500,ea
4,201405,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",500,ea
...,...,...,...,...,...
71290,202109,08910001267,"TR,PIM,FP100R07N3E4,650V,100A,B,INFINEON",550,ea
71291,202110,08910001267,"TR,PIM,FP100R07N3E4,650V,100A,B,INFINEON",550,ea
71292,202111,08910001267,"TR,PIM,FP100R07N3E4,650V,100A,B,INFINEON",550,ea
71293,202112,08910001267,"TR,PIM,FP100R07N3E4,650V,100A,B,INFINEON",550,ea


In [4]:
# 월별+item별 수치 데이터 dataframe 생성
tmp_groupby_date_item = df_inv.groupby(['YYYYMM', 'IEM_CODE']).sum()
tmp_groupby_date_item['YYYYMM'] = [t[0] for t in tmp_groupby_date_item.index]
tmp_groupby_date_item['IEM_CODE'] = [t[1] for t in tmp_groupby_date_item.index]
tmp_groupby_date_item = tmp_groupby_date_item.reset_index(drop=True)
tmp_groupby_date_item = tmp_groupby_date_item[tmp_groupby_date_item.columns.difference(['SPG', '   '])]
tmp_groupby_date_item

,IEM_CODE,OTHER_ISSUE,OTHER_RECEIPT,PO_RECEIPT,SALES_ISSUE,WIP_ISSUE,WIP_RECEIPT,YYYYMM
0,07760000132,16117.0,16114,15750,10164,0.0,0,201401
1,07760000133,15624.0,15622,13680,9142,0.0,0,201401
2,07760000172,3693.0,3691,3000,3631,0.0,0,201401
3,08660003546,0.0,0,0,0,0.0,0,201401
4,08660003547,0.0,0,0,0,0.0,0,201401
...,...,...,...,...,...,...,...,...
52918,10140000088,0.0,0,300,0,300.0,0,202201
52919,11040001102,0.0,0,310,0,310.0,0,202201
52920,646161E4091,4000.0,4000,0,4000,0.0,0,202201
52921,646161E4092,1500.0,1500,0,1500,0.0,0,202201


In [5]:
# merge
merged_data = pd.merge(df_full_date_item, tmp_groupby_date_item, on=["IEM_CODE", "YYYYMM"], how='left').fillna(0)
merged_data = merged_data[['YYYYMM', 'IEM_CODE', 'ITEM_DESCRIPTION', 'PO_RECEIPT', 
                            'SPG', 'UOM', 'OTHER_ISSUE', 'OTHER_RECEIPT', 'SALES_ISSUE', 'WIP_ISSUE', 'WIP_RECEIPT']]

# YYYYMM -> YYYY-MM(datetime)
merged_data['ds'] = [datetime.strptime(str(t), '%Y%m') for t in merged_data['YYYYMM']]

merged_data.to_csv('../../data/processed/INV_Transacton_Summary_merge_filled_0.csv')
merged_data

,YYYYMM,IEM_CODE,ITEM_DESCRIPTION,PO_RECEIPT,SPG,UOM,OTHER_ISSUE,OTHER_RECEIPT,SALES_ISSUE,WIP_ISSUE,WIP_RECEIPT,ds
0,201401,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",3528.0,500,ea,1980.0,1960.0,1960.0,160.0,0.0,2014-01-01
1,201402,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",1568.0,500,ea,1862.0,1862.0,1862.0,0.0,0.0,2014-02-01
2,201403,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",2744.0,500,ea,1568.0,1568.0,1568.0,0.0,0.0,2014-03-01
3,201404,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",2450.0,500,ea,2450.0,2450.0,2450.0,96.0,0.0,2014-04-01
4,201405,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",392.0,500,ea,2254.0,2254.0,2254.0,20.0,0.0,2014-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...
71290,202109,08910001267,"TR,PIM,FP100R07N3E4,650V,100A,B,INFINEON",0.0,550,ea,0.0,6.0,0.0,6.0,0.0,2021-09-01
71291,202110,08910001267,"TR,PIM,FP100R07N3E4,650V,100A,B,INFINEON",0.0,550,ea,0.0,0.0,0.0,0.0,0.0,2021-10-01
71292,202111,08910001267,"TR,PIM,FP100R07N3E4,650V,100A,B,INFINEON",0.0,550,ea,0.0,0.0,0.0,0.0,0.0,2021-11-01
71293,202112,08910001267,"TR,PIM,FP100R07N3E4,650V,100A,B,INFINEON",0.0,550,ea,0.0,0.0,0.0,0.0,0.0,2021-12-01
